# Pràctica 4

In [8]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")

In [9]:
dataset = dataset['train']

# i want to see the first 10 examples
for i in range(10):
    print(dataset[i])
    print()

{'text': 'Reduïu els costos dels processos administratius al vostre organisme públic\nEviteu els desplaçaments i pèrdua de temps als ciutadans en les seves gestions\nOferiu una administració més transparent a ciutadans i empreses\nEns grans i petits experimenten aquesta transformació amb èxit, gràcies al suport de l\'AOC\nDepartament de Sistemes d\'Informació i Processos\n" Via Oberta ens ha permès fer efectiu el dret dels ciutadans a no aportar documents, eliminant paper i simplificant procediments"\n" e.FACT proporciona informació indispensable per a la realització de les auditories del registre comptable de factures de les Administracions Públiques Catalanes"\nCoordinador del departament d\'Informàtica\n"El servei VIA OBERTA és el que ha aportat majors avantatges per als ciutadans"\n"Amb l\' e-NOTUM hem escurçat els procediments en 12 dies, quasi un 40% menys!"\nCoordinadora d\'organització de persones i e-administració\n" Via Oberta ofereix millores per als ciutadans al no haver d\

Obtenim diferents mides de datasets

In [10]:
# i want to get the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:
        print(i)
        break

dataset_100MB = dataset.select(list(range(i)))


# i want to get the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 500000000:
        print(j)
        break

dataset_500MB = dataset.select(list(range(j)))


# i want to get the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 1000000000:
        print(g)
        break

dataset_1GB = dataset.select(list(range(g)))

57333
369003
738007


## Preprocessament

In [11]:
import gensim
from gensim.models import Word2Vec
import re

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    return words

# Carrega el fitxer de text i processa'l
corpus_100MB = []
for line in dataset_100MB:
    words = preprocess_text(line['text'])
    if words:  # Assegura't que la línia no està buida
        corpus_100MB.append(words)

# Comprova algunes línies per assegurar-te que tot està bé
print(corpus_100MB[:2])


[['reduïu', 'els', 'costos', 'dels', 'processos', 'administratius', 'al', 'vostre', 'organisme', 'públic', 'eviteu', 'els', 'desplaçaments', 'i', 'pèrdua', 'de', 'temps', 'als', 'ciutadans', 'en', 'les', 'seves', 'gestions', 'oferiu', 'una', 'administració', 'més', 'transparent', 'a', 'ciutadans', 'i', 'empreses', 'ens', 'grans', 'i', 'petits', 'experimenten', 'aquesta', 'transformació', 'amb', 'èxit', 'gràcies', 'al', 'suport', 'de', 'l', 'aoc', 'departament', 'de', 'sistemes', 'd', 'informació', 'i', 'processos', 'via', 'oberta', 'ens', 'ha', 'permès', 'fer', 'efectiu', 'el', 'dret', 'dels', 'ciutadans', 'a', 'no', 'aportar', 'documents', 'eliminant', 'paper', 'i', 'simplificant', 'procediments', 'e', 'fact', 'proporciona', 'informació', 'indispensable', 'per', 'a', 'la', 'realització', 'de', 'les', 'auditories', 'del', 'registre', 'comptable', 'de', 'factures', 'de', 'les', 'administracions', 'públiques', 'catalanes', 'coordinador', 'del', 'departament', 'd', 'informàtica', 'el', 's

## Entrenament model de similitud de Text Semàntic

In [12]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

In [13]:
from gensim.models import word2vec
model_100MB = word2vec.Word2Vec(corpus_100MB, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model_100MB.save("model_100MB")
# Obtenir un word-vector

In [14]:
model_100MB = Word2Vec.load("model_100MB")

In [15]:
# Exemple: trobar paraules similars
similar_words = model_100MB.wv.most_similar('paraula')
print(similar_words)

[('frase', 0.7188398241996765), ('grafia', 0.6529539227485657), ('lletra', 0.6300726532936096), ('locució', 0.5698095560073853), ('conjunció', 0.5688847303390503), ('recomanació', 0.5602758526802063), ('mania', 0.5528755187988281), ('mot', 0.5462677478790283), ('idea', 0.535517156124115), ('afirmació', 0.534785807132721)]


In [16]:
text_semantic_train = text_semantic['train']

In [17]:
text_semantic_train[0]

{'sentence1': 'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'sentence2': 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 'label': 3.5}

In [76]:
# Preprcessament del text_semantic
corpus_semantic = []
semantic_score = []
for line in text_semantic_train:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic.append((frase1, frase2))
    semantic_score.append(line['label'])

In [19]:
vocabulari = {}

for frases in corpus_semantic:
    for frase in frases:
        for paraula in frase:
            if paraula in vocabulari:
                vocabulari[paraula] += 1
            else:
                vocabulari[paraula] = 1


In [20]:
vocabulari_reduit = {palabra for palabra, frecuencia in vocabulari.items() if frecuencia >= 10}


### Model Baseline

In [23]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 100, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 16)   │     16,000 │ input_layer[0][0… │
│ (Embedding)         │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 100)       │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 100)       │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     20,736 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0],  │
│                     │                   │            │ embedding[1][0],  │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm[1][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

### 1. One-Hot

In [24]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 764, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 764)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 764)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 764, 16)   │     16,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 764)       │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 764)       │          0 │ input_layer_3[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     20,736 │ embedding_1[0][0… │
│                     │                   │            │ not_equal_2[0][0… │
│                     │                   │            │ embedding_1[1][0… │
│                     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128)       │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ lstm_1[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
import tensorflow as tf
def build_and_compile_model(hidden_size: int = 64) -> tf.keras.Model:
  model = tf.keras.Sequential([
      tf.keras.layers.Concatenate(axis=-1, ),
      tf.keras.layers.Dense(hidden_size, activation='relu'),
      tf.keras.layers.Dense(1)
  ])
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ concatenate_3 (Concatenate)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [88]:
import numpy as np

# Función para calcular la representación one-hot de un texto
def compute_one_hot_encoding(text, vocabulari):
    """
    Genera la representación one-hot de un texto utilizando un vocabulario predefinido.
    """
    one_hot_vector = np.zeros(len(vocabulari))
    vocabulario_list = list(vocabulari)
    for palabra in text:
        if palabra in vocabulari:
            index = vocabulario_list.index(palabra)
            one_hot_vector[index] = 1
        
    return one_hot_vector


In [89]:
# Passar les frases de corpus_semantic a vectors
one_hot_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(one_hot_corpus_semantic):
    one_hot_corpus_semantic[i] = (compute_one_hot_encoding(frase1, vocabulari_reduit), 
                                    compute_one_hot_encoding(frase2, vocabulari_reduit))

In [90]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in one_hot_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in one_hot_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 1)
vectors_2 = np.clip(vectors_2, 0, 1)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 1.8048 - val_loss: 0.8186
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.8176 - val_loss: 0.7666
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.6937 - val_loss: 0.7566
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.6056 - val_loss: 0.7558
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.5516 - val_loss: 0.7536
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.4925 - val_loss: 0.7498
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.4901 - val_loss: 0.7520
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4624 - val_loss: 0.7587
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4160 - val_loss: 0.7496
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.3992 - val_loss: 0.7598
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.3765 - val_loss: 0.7684
Epoch 12/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3482 

In [ ]:
# Ejemplo de dos nuevas frases a predecir
new_sentence1 = "Hola "
new_sentence2 = "This is another new sentence"

# Convertir las frases a representaciones one-hot
new_vector1 = compute_one_hot_encoding(new_sentence1, vocabulari_reduit)
new_vector2 = compute_one_hot_encoding(new_sentence2, vocabulari_reduit)

# Hacer la predicción de similitud
similarity_prediction = baseline_model.predict([[new_vector1], [new_vector2]])

print("Similitud predicha entre las dos frases:", similarity_prediction[0][0])


### 2. Models Word2Vec pre-entrenats

In [28]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 100, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 100, 16)   │     16,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_4         │ (None, 100)       │          0 │ input_layer_4[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_5         │ (None, 100)       │          0 │ input_layer_5[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 64)        │     20,736 │ embedding_2[0][0… │
│                     │                   │            │ not_equal_4[0][0… │
│                     │                   │            │ embedding_2[1][0… │
│                     │                   │            │ not_equal_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 128)       │          0 │ lstm_2[0][0],     │
│ (Concatenate)       │                   │            │ lstm_2[1][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         65 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,057 (176.00 KB)

 Trainable params: 45,057 (176.00 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Word2Vec + Mean 
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector


# Word2Vec + Mean Ponderada
def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [33]:
# Passar les frases de corpus_semantic a vectors
mean_embbeding_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(mean_embbeding_corpus_semantic):
    mean_embbeding_corpus_semantic[i] = (compute_mean_embedding(frase1, model_100MB.wv, 100), 
                                        compute_mean_embedding(frase2, model_100MB.wv, 100))

In [34]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in mean_embbeding_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in mean_embbeding_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 999)
vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

(2073, 100)
(2073, 100)
(2073,)
Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - loss: 0.9921 - val_loss: 0.6544
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.6827 - val_loss: 0.7196
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.7162 - val_loss: 0.6233
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - loss: 0.6877 - val_loss: 0.6375
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.6734 - val_loss: 0.6852
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.6844 - val_loss: 0.6304
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.6565 - val_loss: 0.6417
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - loss: 0.6670 - val_loss: 0.6827
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.6817 - val_loss: 0.6352
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.6733 - val_loss: 0.6240
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.6502 - val_loss: 0.6228
Epoch 12/50
52/52 ━━━━━━━━━━━━

### 3. spaCy

In [35]:
!py -m spacy download ca_core_news_md

     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/49.2 MB ? eta -:--:--
     --------------------------------------- 0.0/49.2 MB 217.9 kB/s eta 0:03:46
     --------------------------------------- 0.0/49.2 MB 245.8 kB/s eta 0:03:21
     --------------------------------------- 0.1/49.2 MB 599.1 kB/s eta 0:01:22
     ---------------------------------------- 0.4/49.2 MB 1.8 MB/s eta 0:00:27
      --------------------------------------- 0.9/49.2 MB 3.3 MB/s eta 0:00:15
     - -------------------------------------- 2.0/49.2 MB 6.2 MB/s eta 0:00:08
     -- ------------------------------------- 2.9/49.2 MB 8.0 MB/s eta 0:00:06
     --- ------------------------------------ 3.8/49.2 MB 9.4 MB/s eta 0:00:05
     --- ------------------------------------ 4.8/49.2 MB 10.6 MB/s eta 0:00:05
     --- ------------------------------------ 4.9/49.2 MB 10.4 MB/s e

In [36]:
import spacy

nlp_md = spacy.load('ca_core_news_md')

In [48]:
def spacy_mean_embedding(text1, text2, model):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    text1 = ' '.join(text1)
    text2 = ' '.join(text2)
    frase1 = model(text1).vector
    frase2 = model(text2).vector
    
    return frase1, frase2

In [49]:
# Passar les frases de corpus_semantic a vectors
spacy_mean_embbeding_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_mean_embbeding_corpus_semantic):
    
    frase1, frase2 = spacy_mean_embedding(frase1, frase2, nlp_md)
    spacy_mean_embbeding_corpus_semantic[i] = (frase1, frase2)

In [85]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in spacy_mean_embbeding_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in spacy_mean_embbeding_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

(2073, 300)
(2073, 300)
(2073,)
Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 348ms/step - loss: 0.9142 - val_loss: 0.6697
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 14s 263ms/step - loss: 0.6794 - val_loss: 0.6241
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 15s 288ms/step - loss: 0.6562 - val_loss: 0.6427
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 380ms/step - loss: 0.6672 - val_loss: 0.6415
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 15s 266ms/step - loss: 0.6285 - val_loss: 0.6420
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 25s 355ms/step - loss: 0.6285 - val_loss: 0.6506
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 375ms/step - loss: 0.6226 - val_loss: 0.6543
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 25s 478ms/step - loss: 0.6036 - val_loss: 0.6429
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 378ms/step - loss: 0.6031 - val_loss: 0.6561
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 401ms/step - loss: 0.6169 - val_loss: 0.6420
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 374ms/step - loss: 0.5954 - val_loss: 0.6749
Epoch 12/

#### 4.RoBERTa

In [62]:
!py -m spacy download ca_core_news_trf

     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/457.1 MB ? eta -:--:--
     -------------------------------------- 0.0/457.1 MB 262.6 kB/s eta 0:29:01
     -------------------------------------- 0.1/457.1 MB 363.1 kB/s eta 0:20:59
     ---------------------------------------- 0.2/457.1 MB 1.0 MB/s eta 0:07:22
     ---------------------------------------- 0.5/457.1 MB 2.0 MB/s eta 0:03:48
     ---------------------------------------- 1.0/457.1 MB 3.8 MB/s eta 0:02:01
     ---------------------------------------- 1.9/457.1 MB 5.8 MB/s eta 0:01:18
     ---------------------------------------- 2.8/457.1 MB 7.5 MB/s eta 0:01:01
     ---------------------------------------- 3.6/457.1 MB 8.6 MB/s eta 0:00:53
     ---------------------------------------- 4.6/457.1 MB 9.8 MB/s eta 0:00:47
     --------------------------------------- 5.5/457.1 MB 10.

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [65]:
import spacy

nlp_trf = spacy.load('ca_core_news_trf')

ValueError: [E002] Can't find factory for 'curated_transformer' for language Catalan (ca). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, ca.lemmatizer

### 5. RoBERTa fine-tuned

In [67]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [69]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

In [82]:
fine_runed_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic[i] = (frase1, frase2)


In [83]:
predictions = pipe(prepare(fine_runed_corpus_semantic), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)

[{'label': 'SIMILARITY', 'score': 3.639443974101274}, {'label': 'SIMILARITY', 'score': 1.3343049254405437}, {'label': 'SIMILARITY', 'score': 3.6255003414699982}, {'label': 'SIMILARITY', 'score': 2.8571212701413073}, {'label': 'SIMILARITY', 'score': 1.7412681993979517}, {'label': 'SIMILARITY', 'score': 2.902458331608203}, {'label': 'SIMILARITY', 'score': 3.3925175786892536}, {'label': 'SIMILARITY', 'score': 2.7214751443641485}, {'label': 'SIMILARITY', 'score': 2.848001481281147}, {'label': 'SIMILARITY', 'score': 2.8075600107730767}, {'label': 'SIMILARITY', 'score': 2.9737861311233997}, {'label': 'SIMILARITY', 'score': 0.946479625573919}, {'label': 'SIMILARITY', 'score': 2.0864164114103474}, {'label': 'SIMILARITY', 'score': 3.390759591752991}, {'label': 'SIMILARITY', 'score': 3.119538929859854}, {'label': 'SIMILARITY', 'score': 2.835003453924068}, {'label': 'SIMILARITY', 'score': 1.6847840009938235}, {'label': 'SIMILARITY', 'score': 3.1470772247192316}, {'label': 'SIMILARITY', 'score': 2

### model amb embeddings entrenables

#### Random Embeddings

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

vectors_1 =[pair[0] for pair in mean_embbeding_corpus_semantic]
vectors_2 =[pair[1] for pair in mean_embbeding_corpus_semantic]

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings aleatoris
random_embeddings = np.random.uniform(-1, 1, (vocab_size, embedding_dim))

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

# Entrenament del model amb embeddings aleatoris
model_random = build_and_compile_model(random_embeddings, input_length=max_length, hidden_size=hidden_size)
model_random.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_random = model_random.evaluate([data_1, data_2], labels)
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Random Embeddings - Loss: {loss_random}')
print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')


#### Word2Vec

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Exemple de dades
texts_1 = ["aquest és un text d'exemple", "un altre text"]
texts_2 = ["aquest és un altre text", "un text diferent"]
labels = [0.8, 0.4]  # Similitud entre les frases

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

def build_and_compile_model(embedding_matrix, input_length: int = 100, hidden_size: int = 64) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)

    # Define Layers
    embedding = tf.keras.layers.Embedding(
        vocab_size, embedding_dim, weights=[embedding_matrix], input_length=input_length, mask_zero=True, trainable=True)

    lstm = tf.keras.layers.LSTM(hidden_size)

    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)

    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)

    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')


### Prova

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Defineix funcions per calcular els embeddings de frases
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector

def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [ ]:
# Exemple de corpus
corpus = ["Aquest és un text de exemple", "Aquí hi ha un altre text"]

# Prepara els vectors TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [ ]:
# Exemple d'ús amb embeddings pre-entrenats (Word2Vec)
from gensim.models import KeyedVectors
import tensorflow as tf

# Carrega el model pre-entrenat (substitueix 'path_to_model' amb la ruta real)
# word2vec_model = KeyedVectors.load_word2vec_format('path_to_model', binary=True)

# Defineix la longitud de les seqüències i la dimensió dels embeddings
input_length = 10
embedding_dim = model_100MB.vector_size

# Prepara les dades d'entrenament
X_train1 = np.array([compute_mean_embedding(text.split(), model_100MB.wv, embedding_dim) for text in corpus])
X_train2 = np.array([compute_weighted_mean_embedding(text.split(), model_100MB.wv, word2tfidf, embedding_dim) for text in corpus])
y_train = np.random.rand(len(corpus), 1)

# Defineix el model de Keras (ajusta les dimensions segons la longitud de les seqüències i la dimensió dels embeddings)
def build_and_compile_model(input_dim, hidden_size=64):
    input_1 = tf.keras.Input(shape=(input_dim,))
    input_2 = tf.keras.Input(shape=(input_dim,))
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(concatenate)
    output = tf.keras.layers.Dense(1)(hidden)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

# Construir i compilar el model
m = build_and_compile_model(input_dim=embedding_dim)

# Entrenar el model
m.fit([X_train1, X_train2], y_train, epochs=10, batch_size=32)

# Exemple de predicció
input_data1 = compute_mean_embedding("Aquest és un text de exemple".split(), model_100MB.wv, embedding_dim).reshape(1, -1)
input_data2 = compute_weighted_mean_embedding("Aquí hi ha un altre text".split(), model_100MB.wv, word2tfidf, embedding_dim).reshape(1, -1)

# Predir la similitud
y_pred = m.predict([input_data1, input_data2])
print(y_pred)

Epoch 1/10
1/1 [==============================] - 0s 334ms/step - loss: 0.7557
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1112
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4268
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5486
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4800
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3008
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0543
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2087
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.2880
Epoch 10/10
1/1 [==============================] - 0s 55ms/step
[[0.10263795]]


In [ ]:
y_train

array([[0.24829134],
       [0.49559584]])